In [6]:
import numpy as np

# Define the AD9912 parameters
fs = 1.0e9       # Sampling Frequency (Hertz)
N = 48           # Frequency Tuning Word Bits
P = 19           # Phase Truncation Bits
fo = 21.0e6      # Target Output Frequency Center (Hertz)
fsweep = 1e3     # Frequency sweep range about fo (Hertz)
fstepsize = 1e-4 # Frequency stepsize to use during sweep (Hertz)

# Calculate Min and Max Frequency Tuning Word (hex) from fo and fsweep range
startfo = fo - fsweep / 2
endfo = fo + fsweep / 2

startFTW = int( round( 2 ** N * (startfo / fs) ) )
endFTW = int( round( 2 ** N * (endfo / fs) ) )
stepsizeFTW = int( round( 2 ** N * (fstepsize / fs) ) )

print "Start FTW (dec) = ", startFTW
print "End FTW (dec) = ", endFTW
print "Stepsize FTW (dec) = ", stepsizeFTW
    
#Preload Max spur variables for checking
max_PPTMag_L = 0
max_PPTMag = -200.0
max_PPTMag_freq = 0.0

# Loop calculations for each FTW from Start to End of fsweep range
FTW = startFTW
while FTW <= endFTW:

    # Convert Frequency Tuning Word to binary with leading zeros to express them as 48 bits
    FTW2 = (bin (FTW) )[2:] .zfill(48)
    
    # Determine the index of the rightmost non-zero bit "L"
    L = FTW2.rfind('1') + 1 # Index needs to start at 1 instead of 0

    # Calculate PPT Magnitude in dBc
    PPTMag = np.float( 20 * np.log10( (np.sin( np.pi / (2 ** L) )) / (np.sin( (np.pi * (2 ** P - 1)) / (2 ** L) )) ) )
    
    # Calculate PPT Frequencies:
    # Truncate FTW2 to L bits then convert to decimal values to determine K
    Trunc_L_FTW = FTW2[:L]
    
    # Convert the truncated FTW to decimal for K
    K = int(Trunc_L_FTW, 2)
   
    # Calculate the spectral index positions of the two PPT spurs (R1 and R2)
    R1 = np.float( (K * (2 ** P - 1)) % (2 ** L) )
    R2 = np.float( (K * ((2 ** L) - (2 ** P) + 1)) % (2 ** L) )
   
    # Calculate each PPT frequency from fs, R1 and R2
    fPPT1 = np.float( fs * (R1 / (2 ** L)) )
    fPPT2 = np.float( fs * (R2 / (2 ** L)) )

    # Check for to see if spur is the new maximum
    if PPTMag >= max_PPTMag:
        max_PPTMag_L = L
        max_PPTMag = PPTMag
        
        if abs(fo - fPPT1) <= abs(fo - fPPT2):
            max_PPTMag_freq = fPPT1
        else:
            max_PPTMag_freq = fPPT2
    
    FTW += stepsizeFTW
    
print "Maximum PPT Magnitude Calculated =", "{0:.5f}" .format(max_PPTMag), "dBc at", 
print "{0:.3e}" .format(max_PPTMag_freq), "Hz with L =", max_PPTMag_L

Start FTW (dec) =  5910833773435
End FTW (dec) =  5911115248412
Stepsize FTW (dec) =  28

...Calculating...

Maximum PPT Magnitude Calculated = -114.39138 dBc at 2.891e+08 Hz with L = 48
